In [ ]:
import gc
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import janestreet

In [ ]:
data = pd.read_csv('../input/jane-street-market-prediction/train.csv')
data=data[data.weight!=0]


In [ ]:
example_submission = pd.read_csv('../input/jane-street-market-prediction/example_sample_submission.csv')
features = pd.read_csv('../input/jane-street-market-prediction/features.csv')

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
# Settings
NAN_VALUE = -999
features = [c for c in data.columns if 'feature' in c]
target = 'resp'

# Split into features X and target Y
X = data.loc[:, features].fillna(NAN_VALUE)
Y = (data.loc[:, target] > 0).astype(int)

In [ ]:
lgbmc = LGBMClassifier(n_estimators=500)

In [ ]:
lgbmc.fit(X, Y)

In [ ]:
# Clear memory
del X, Y
gc.collect()

In [ ]:
# Create submission
env = janestreet.make_env()
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test: 
    X_test = test_df.loc[:, features]
    X_test = X_test.fillna(NAN_VALUE)
    sample_prediction_df.action = lgbmc.predict(X_test)
    env.predict(sample_prediction_df)
 